In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [3]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [4]:
#Extract zipped folders

import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [5]:
#move images to single destimation folder

import shutil
import os

source_dir1 = '/content/Qualcomm-DL-Hackathon/train/images/images part-1'
source_dir2 = '/content/Qualcomm-DL-Hackathon/train/images/images part-2'
destination_dir = '/content/Qualcomm-DL-Hackathon/dataset/images'
os.makedirs(destination_dir, exist_ok=True)

def copy_files(source, destination):
    for filename in os.listdir(source):
        source_file = os.path.join(source, filename)
        destination_file = os.path.join(destination, filename)
        if os.path.isfile(source_file):
          shutil.copy(source_file, destination_dir)


copy_files(source_dir1, destination_dir)
copy_files(source_dir2, destination_dir)

shutil.rmtree(source_dir1)
shutil.rmtree(source_dir2)

print("Files copied successfully!")

Files copied successfully!


In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [7]:
#Import required modules

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import pandas as pd
import os
import torch.nn.functional as F
import optuna


In [8]:
class EmergencyVehicleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
         #Get path for image
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        #check if CSV file has more than one column. Train CSV has 2nd column to be extracted.
        if self.data.shape[1] > 1:
            label = self.data.iloc[idx, 1]
        else:
            label = 0 #Test CSV doesn't have 2nd column. so, assign 0 by default.

        #Apply transform for images mentioned CSV file
        if self.transform:
            image = self.transform(image)
        return image, label


In [9]:
# Define transformations

#transform with data augmentation.
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Random crop and resize to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),      # Random rotation of up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])


#Data augmentation is degrading model performance. So, applying below transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  #All images are with 224x224. Default is giving better performance
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


In [10]:
# Load the dataset
train_csv_dir = '/content/Qualcomm-DL-Hackathon/train/train.csv'
test_csv_dir = '/content/Qualcomm-DL-Hackathon/test.csv'
train_dataset = EmergencyVehicleDataset(csv_file=train_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])


In [11]:
#Use GPU when available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GoogleNet/Inception model is giving better performance for this dataset. But need to identify best hyperparameters. Using Optuna for this

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Load the dataset with the suggested batch size
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

    # Load the pretrained GoogLeNet model
    model = models.googlenet(pretrained=True)

    # Modify the classifier to match the number of outputs (2 in this case)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)
    model.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Use Adam optimizer

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train() #Set the model to train
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for images, labels in train_loader: #Iterate over batches of images and labels
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad() #Clear the gradients
            outputs = model(images)
            loss = criterion(outputs, labels) #Compute the loss
            loss.backward() #backpropagate the loss to adjust gradients
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train

        # Validation loop
        model.eval() #Set the model to eval
        correct_val = 0
        total_val = 0
        with torch.no_grad():  #Disables gradient calculation for validation
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')

    return val_accuracy

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print('Best hyperparameters:', study.best_params)


[I 2025-01-05 16:48:50,622] A new study created in memory with name: no-name-043a5786-3fac-4aaf-bf37-a64d1daf2e7a
<ipython-input-83-817ca402516c>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


Epoch [1/10], Loss: 0.6309, Training Accuracy: 69.38%, Validation Accuracy: 43.03%
Epoch [2/10], Loss: 0.5985, Training Accuracy: 73.56%, Validation Accuracy: 74.85%
Epoch [3/10], Loss: 0.5458, Training Accuracy: 76.29%, Validation Accuracy: 78.18%
Epoch [4/10], Loss: 0.4321, Training Accuracy: 80.47%, Validation Accuracy: 73.64%
Epoch [5/10], Loss: 0.3903, Training Accuracy: 82.22%, Validation Accuracy: 79.70%
Epoch [6/10], Loss: 0.3645, Training Accuracy: 84.27%, Validation Accuracy: 82.42%
Epoch [7/10], Loss: 0.3558, Training Accuracy: 84.80%, Validation Accuracy: 60.91%
Epoch [8/10], Loss: 0.3737, Training Accuracy: 83.66%, Validation Accuracy: 84.55%
Epoch [9/10], Loss: 0.3432, Training Accuracy: 86.47%, Validation Accuracy: 71.21%


[I 2025-01-05 16:50:08,102] Trial 0 finished with value: 73.33333333333333 and parameters: {'lr': 0.0055284956931205805, 'batch_size': 16}. Best is trial 0 with value: 73.33333333333333.


Epoch [10/10], Loss: 0.2772, Training Accuracy: 87.54%, Validation Accuracy: 73.33%
Epoch [1/10], Loss: 0.5224, Training Accuracy: 76.22%, Validation Accuracy: 76.97%
Epoch [2/10], Loss: 0.5182, Training Accuracy: 77.28%, Validation Accuracy: 73.03%
Epoch [3/10], Loss: 0.6075, Training Accuracy: 72.64%, Validation Accuracy: 45.45%
Epoch [4/10], Loss: 0.5137, Training Accuracy: 76.52%, Validation Accuracy: 75.76%
Epoch [5/10], Loss: 0.4336, Training Accuracy: 81.16%, Validation Accuracy: 57.58%
Epoch [6/10], Loss: 0.3779, Training Accuracy: 83.59%, Validation Accuracy: 76.36%
Epoch [7/10], Loss: 0.3331, Training Accuracy: 85.94%, Validation Accuracy: 78.48%
Epoch [8/10], Loss: 0.3176, Training Accuracy: 87.54%, Validation Accuracy: 79.70%
Epoch [9/10], Loss: 0.2855, Training Accuracy: 87.99%, Validation Accuracy: 43.94%


[I 2025-01-05 16:51:25,248] Trial 1 finished with value: 86.06060606060606 and parameters: {'lr': 0.005693326776517646, 'batch_size': 16}. Best is trial 1 with value: 86.06060606060606.


...

Epoch [10/10], Loss: 0.0099, Training Accuracy: 99.92%, Validation Accuracy: 92.73%
Epoch [1/10], Loss: 0.5413, Training Accuracy: 77.81%, Validation Accuracy: 85.15%
Epoch [2/10], Loss: 0.3330, Training Accuracy: 91.72%, Validation Accuracy: 90.30%
Epoch [3/10], Loss: 0.2205, Training Accuracy: 93.92%, Validation Accuracy: 90.91%
Epoch [4/10], Loss: 0.1535, Training Accuracy: 96.35%, Validation Accuracy: 92.42%
Epoch [5/10], Loss: 0.1080, Training Accuracy: 97.57%, Validation Accuracy: 92.73%
Epoch [6/10], Loss: 0.0807, Training Accuracy: 98.71%, Validation Accuracy: 92.12%
Epoch [7/10], Loss: 0.0644, Training Accuracy: 98.94%, Validation Accuracy: 92.73%
Epoch [8/10], Loss: 0.0436, Training Accuracy: 99.70%, Validation Accuracy: 93.33%
Epoch [9/10], Loss: 0.0334, Training Accuracy: 99.77%, Validation Accuracy: 92.42%


[I 2025-01-05 17:13:16,720] Trial 18 finished with value: 93.03030303030303 and parameters: {'lr': 2.8469938225034774e-05, 'batch_size': 64}. Best is trial 15 with value: 94.24242424242425.


Epoch [10/10], Loss: 0.0253, Training Accuracy: 99.77%, Validation Accuracy: 93.03%
Epoch [1/10], Loss: 0.2690, Training Accuracy: 89.13%, Validation Accuracy: 93.64%
Epoch [2/10], Loss: 0.0666, Training Accuracy: 97.95%, Validation Accuracy: 90.30%
Epoch [3/10], Loss: 0.0252, Training Accuracy: 99.47%, Validation Accuracy: 92.42%
Epoch [4/10], Loss: 0.0177, Training Accuracy: 99.39%, Validation Accuracy: 93.64%
Epoch [5/10], Loss: 0.0171, Training Accuracy: 99.62%, Validation Accuracy: 89.39%
Epoch [6/10], Loss: 0.0455, Training Accuracy: 98.56%, Validation Accuracy: 91.52%
Epoch [7/10], Loss: 0.0339, Training Accuracy: 98.63%, Validation Accuracy: 93.94%
Epoch [8/10], Loss: 0.0125, Training Accuracy: 99.62%, Validation Accuracy: 93.64%
Epoch [9/10], Loss: 0.0065, Training Accuracy: 99.85%, Validation Accuracy: 88.79%


[I 2025-01-05 17:14:34,148] Trial 19 finished with value: 92.72727272727273 and parameters: {'lr': 0.0008575928390375985, 'batch_size': 64}. Best is trial 15 with value: 94.24242424242425.


Epoch [10/10], Loss: 0.0106, Training Accuracy: 99.70%, Validation Accuracy: 92.73%
Best hyperparameters: {'lr': 0.00022681637981194816, 'batch_size': 64}


Best hyperparameters identified as below: {'lr': 0.00022681637981194816, 'batch_size': 64}.
We need to identify best num_epochs  to not overfit

In [12]:
# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [13]:
import torchvision.models as models

# GoogleNet model
model = models.googlenet(pretrained=True)

# Modify the fully connected layer for your specific task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00022681637981194816)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 125MB/s]


In [14]:
# Train the model
previous_loss = 0.0
num_epochs = 20
for epoch in range(num_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # Validation loop
    model.eval() #Set the model to eval
    correct_val = 0
    total_val = 0
    with torch.no_grad():  #Disables gradient calculation for validation
        for images, labels in val_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_val += labels.size(0)
          correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')


Epoch [1/20], Loss: 0.2995, Training Accuracy: 86.70%, Validation Accuracy: 94.85%
Epoch [2/20], Loss: 0.0758, Training Accuracy: 97.19%, Validation Accuracy: 95.76%
Epoch [3/20], Loss: 0.0233, Training Accuracy: 99.54%, Validation Accuracy: 95.15%
Epoch [4/20], Loss: 0.0094, Training Accuracy: 99.77%, Validation Accuracy: 94.85%
Epoch [5/20], Loss: 0.0100, Training Accuracy: 99.70%, Validation Accuracy: 95.76%
Epoch [6/20], Loss: 0.0068, Training Accuracy: 99.70%, Validation Accuracy: 96.06%
Epoch [7/20], Loss: 0.0052, Training Accuracy: 99.77%, Validation Accuracy: 96.06%
Epoch [8/20], Loss: 0.0066, Training Accuracy: 99.70%, Validation Accuracy: 95.45%
Epoch [9/20], Loss: 0.0055, Training Accuracy: 99.62%, Validation Accuracy: 95.76%
Epoch [10/20], Loss: 0.0056, Training Accuracy: 99.70%, Validation Accuracy: 95.76%
Epoch [11/20], Loss: 0.0053, Training Accuracy: 99.70%, Validation Accuracy: 95.76%
Epoch [12/20], Loss: 0.0048, Training Accuracy: 99.70%, Validation Accuracy: 96.06%
E

As seen from above, 6-10 epochs should be enough to train the model. Now, let's build the model with all training datasets without validation set. Loss should be less

In [39]:
import torchvision.models as models

# GoogleNet model
model = models.googlenet(pretrained=True)

# Modify the fully connected layer for your specific task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00022681637981194816)


# Create train DataLoader with complete dataset
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss : .4f}, Accuracy: {train_accuracy:.2f}%')
    if epoch_loss < 0.02:
      print('Required loss reached')
      break

print('Finished Training')

Epoch [1/20], Loss:  0.2641, Accuracy: 88.21%
Epoch [2/20], Loss:  0.0632, Accuracy: 97.69%
Epoch [3/20], Loss:  0.0177, Accuracy: 99.57%
Required loss reached
Finished Training


In [40]:
test_dataset = EmergencyVehicleDataset(csv_file=test_csv_dir, root_dir='/content/Qualcomm-DL-Hackathon/dataset/images', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Make predictions on the test data
predictions = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a submission dataframe
submission_df = pd.DataFrame({'image_names': test_data['image_names'], 'emergency_or_not': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv


In [42]:
#Save the model
torch.save(model, 'entire_model.pth')